#### First let's import a few things

In [ ]:
import os
import ceci
from ceci.pipeline import Pipeline

#### For now we need to be in the top-level ceci directory to read the input yaml file

This is b/c it refers to the configuration file, which we will also need to read.

We should fix this.

In [ ]:
os.chdir(os.path.join(os.path.dirname(ceci.__file__), '..'))
pipeline = Pipeline.read('tests/test.yml')

#### Now we have some pipeline introspection

1. Printing a list of stages
2. Accessing a stage as an attribute of the pipeline
3. Printing the inputs and output connections associated to a stage
4. Printing the configuration associated to a stage

In [ ]:
pipeline.print_stages()

In [ ]:
print(pipeline.SysMapMaker)

In [ ]:
pipeline.SysMapMaker.print_io()

In [ ]:
print(repr(pipeline.WLGCSelector.config))

#### Now we show how to save and read back a pipeline

The two different versions (i.e., with and without the reduce_config=True) will save slightly different versions
of the yaml file.  

In the former case we explicitly save everything for each stage of the pipeline.
In the later case, we reduce the configuration by parsing out the inputs, outputs and global params.  

In [ ]:
pipeline.save('test_save.yml')

In [ ]:
pipe_read = Pipeline.read('test_save.yml')

In [ ]:
pipeline.save('test_save_v2.yml', reduce_config=True)

In [ ]:
pipe_read_v2 = Pipeline.read('test_save_v2.yml')

#### Ok, eventually this example should include most stuff, but for now let's just the pipeline

In [ ]:
status = pipeline.run()

#### Now let's do the same thing by building the pipeline with the python interface

In [ ]:
from ceci_example.example_stages import *

In [ ]:
pipe2 = Pipeline.interactive()
overall_inputs = {'DM':'./tests/inputs/dm.txt',
                  'fiducial_cosmology':'./tests/inputs/fiducial_cosmology.txt'}
inputs = overall_inputs.copy()
inputs['metacalibration'] = True
inputs['config'] = None

pipe2.pipeline_files.update(**inputs)
pipe2.build_stage(PZEstimationPipe)
pipe2.build_stage(shearMeasurementPipe, apply_flag=False)
pipe2.build_stage(WLGCSelector, zbin_edges=[0.2, 0.3, 0.5], ra_range=[-5, 5])
pipe2.build_stage(SysMapMaker)
pipe2.build_stage(SourceSummarizer)
pipe2.build_stage(WLGCCov)
pipe2.build_stage(WLGCRandoms)
pipe2.build_stage(WLGCTwoPoint)
pipe2.build_stage(WLGCSummaryStatistic)


In [ ]:
pipe2.WLGCCov.config

In [ ]:
pipe2.initialize(overall_inputs, pipeline.run_config, pipeline.stages_config)

In [ ]:
pipe2.WLGCCov.outputs

In [ ]:
pipe2.WLGCCov.config['aliases'] = dict(covariance='covariance_copy')
pipe2.WLGCSummaryStatistic.config['aliases'] = dict(covariance='covariance_copy')

In [ ]:
pipe2.WLGCCov.find_outputs('.')

In [ ]:
pipe2.run()